In [1]:
import logging

logger = logging.getLogger(__name__)

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from short_tracker.data import (
    query_all_sec_metadata, query_uk_si_disclosures, query_mkt_data, query_quotes,
    SHORT_URL_UK, DATE_COL, FUND_COL, ISIN_COL, SHORT_POS_COL,
    SHARE_ISSUER_COL, UK_DISCL_THRESHOLD,
)
from short_tracker.processing import (
    check_cur_hist_discl_overlap, remove_dupl_shorts, ffill_discl_data, calc_fund_short_flow_bounds,
    extract_sec_tickers, process_mkt_data, subset_top_shorts,
)

NUM_SUBSET = 20
UK_MKT_TICKER = "VUKE"
ALPHAVANGATE_KEY = ""

In [3]:
discl_data, rept_date = query_uk_si_disclosures(SHORT_URL_UK) # exp_upd_time
rept_date

datetime.date(2022, 12, 30)

In [4]:
cur_discl = discl_data['current']

exp_max_discl_date = rept_date - timedelta(days=1)
max_discl_date = cur_discl[DATE_COL].max().date()

assert max_discl_date <= exp_max_discl_date
max_discl_date

datetime.date(2022, 12, 29)

In [5]:
# subset on current top N total overall shorts + top N individual shorts
top_sec_shorts, top_fund_shorts = subset_top_shorts(cur_discl, NUM_SUBSET)

In [6]:
isins = list({*top_fund_shorts[ISIN_COL].values, *top_sec_shorts[ISIN_COL].values})
print(len(isins))
isins[:5]

30


['GB00BK7YQK64',
 'GB00B0H2K534',
 'GB0001638955',
 'GB00B63QSB39',
 'GB00B7KR2P84']

In [7]:
sec_metadata, err_isins = query_all_sec_metadata(isins, {"idType": "ID_ISIN", "exchCode": "LN"})
len(err_isins)

0

In [8]:
isin_ticker_map = extract_sec_tickers(sec_metadata)
tickers = [UK_MKT_TICKER, *isin_ticker_map.values()]
query_tickers = [k.rstrip("/") + '.L' for k in tickers]

ticker_map = dict(zip(tickers, query_tickers))

tickers[:5]

['VUKE', 'HMSO', 'PFC', 'JDW', 'GRG']

In [10]:
query_start = datetime.today().date() - timedelta(days=365)

mkt_data = {tkr: query_mkt_data(qry_tkr, query_start) for tkr, qry_tkr in ticker_map.items()}
quotes = {tkr: query_quotes(qry_tkr) for tkr, qry_tkr in ticker_map.items()}

In [11]:
SH_OUT_COL = 'shares_outstanding'

mkt_data_df = process_mkt_data(mkt_data)
quotes_df = pd.DataFrame(quotes)
sh_out = quotes_df.loc['sharesOutstanding'].rename(SH_OUT_COL)

missing_sh_out = list(sh_out[sh_out.isna()].index)

if missing_sh_out:
    logger.warning(f"No share outstanding data for tickers: {missing_sh_out}")

No share outstanding data for tickers: ['VUKE']


In [12]:
mkt_data_df

,Close,Adj Close,Volume,Ticker
2021-12-30,32.334999,32.334999,181089,VUKE
2021-12-31,32.259998,32.259998,63784,VUKE
2022-01-04,32.775002,32.775002,922919,VUKE
2022-01-05,32.860001,32.860001,379070,VUKE
2022-01-06,32.570000,32.570000,505078,VUKE
...,...,...,...,...
2022-12-22,34.950001,34.950001,10006476,BOO
2022-12-23,34.750000,34.750000,9713914,BOO
2022-12-28,33.889999,33.889999,13647584,BOO
2022-12-29,36.000000,36.000000,9983660,BOO


In [13]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
ALPHAVANTAGE_ENDPOINT = "https://www.alphavantage.co/query?"

import requests

from short_tracker.data import APIRateLimitException

# def query_mkt_data(ticker, api_key):
#     """Query Alphavantage for adjusted stock prices and """
#     def query_endpoint(fn):
#         av_args = {"function": fn, "symbol": ticker, "apikey": api_key}
#         resp = requests.get(ALPHAVANTAGE_ENDPOINT, params=av_args)
#         resp.raise_for_status()
#         data = resp.json()
        
#         if "Note" in data:
#             raise APIRateLimitException
#         elif "Error Message" in data:
#             raise ValueError(data["Error Message"])
#         return data
        
#     price = query_endpoint("TIME_SERIES_DAILY_ADJUSTED")
#     sec_info = query_endpoint("OVERVIEW")
#     return price, sec_info

# price, sec_info = query_mkt_data("RWI.L", ALPHAVANGATE_KEY)